In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.ticker as mticker
import time
import csv
import zoneinfo
import matplotlib.pyplot as plt
import matplotlib.colors
import math

In [2]:
data_fn = '../data/google_reviews_hourly_cut_top.csv'
df = pd.read_csv(data_fn, index_col=1)
display(df)

category  1550934000.0  1550937600.0  \
1550930400.0                                                                
32                Air conditioning contractor            38            45   
32            Air conditioning repair service            31            54   
2152                      American restaurant          2535          2541   
70                           Amusement center            72            76   
70                            Animal hospital            76            99   
...                                       ...           ...           ...   
91                    Wheel alignment service           129           142   
69                     Window treatment store            92           125   
167                                  Wine bar           196           184   
80                                 Wine store            96           136   
237                    Women's clothing store           308           310   

              1550941200.0  1550944800.0  1550948400.0  1550952000.0  \
1550930400.0                                                           
32                      60            49            42            48   
32                      51            50            44            46   
2152                  2851          3155          3493          3542   
70                      86           120           159           175   
70                      89            71            71            65   
...                    ...           ...           ...           ...   
91                     125           147           122           151   
69                     142           152           185           172   
167                    201           222           239           281   
80                     151           158           164           193   
237                    416           490           532           612   

              1550955600.0  1550959200.0  1550962800.0  ...  1696024800.0  \
1550930400.0                                            ...                 
32                      49            43            27  ...             0   
32                      49            37            29  ...             0   
2152                  3524          3753          3909  ...             0   
70                     223           260           255  ...             0   
70                      57            60            82  ...             0   
...                    ...           ...           ...  ...           ...   
91                     126           117           102  ...             0   
69                     179           191           173  ...             0   
167                    260           324           361  ...             0   
80                     179           216           219  ...             0   
237                    690           684           674  ...             0   

              1696028400.0  1696032000.0  1696035600.0  1696039200.0  \
1550930400.0                                                           
32                       0             0             0             0   
32                       0             0             0             0   
2152                     0             0             0             0   
70                       0             0             0             0   
70                       0             0             0             0   
...                    ...           ...           ...           ...   
91                       0             0             0             0   
69                       0             0             0             0   
167                      0             0             0             0   
80                       0             0             0             0   
237                      0             0             0             0   

              1696042800.0  1696046400.0  1696050000.0  1696053600.0  \
1550930400.0                                                           
32             

In [49]:
start_date = datetime.datetime(2021, 1, 1, 00, 00).timestamp()
print(start_date)
print(list(df.columns).index(str(start_date)))

1609488000.0
16266


In [55]:
timeseries = df.to_numpy()
# remove columns
end_timestamp = list(df.columns).index(str(datetime.datetime(2021, 6, 1, 00, 00).timestamp()))
timeseries = timeseries[:,1:end_timestamp]
# remove zero rows
timeseries = timeseries[~np.all(timeseries == 0, axis=1)]

time_interval = 60*60
time_arr = np.array([datetime.datetime.utcfromtimestamp(int(float(timestamp))).strftime('%m-%d-%Y') for timestamp in df.columns[1:end_timestamp]])
years = np.array([datetime.datetime.utcfromtimestamp(int(float(timestamp))).strftime('%Y') for timestamp in df.columns[1:end_timestamp]])
months = np.array([datetime.datetime.utcfromtimestamp(int(float(timestamp))).strftime('%m') for timestamp in df.columns[1:end_timestamp]])
weekdays_timestamps = np.array([datetime.datetime.utcfromtimestamp(int(float(timestamp))).weekday() for timestamp in df.columns[1:end_timestamp]])
timeseries_len = timeseries.shape[1]

categories = df['category'].to_numpy()

print(timeseries.shape)

(205, 19888)


In [53]:
scale = 1000
scale_ind = 10
years_cut = years[::scale_ind]
time_arr_cut = time_arr[::scale_ind]

# make figure + axes
fig, ax = plt.subplots(tight_layout=True)
ax.set_xlabel('date')
ax.set_ylabel(f'count (x{scale})')
plt.title("Weekly Google Reviews by Category")

rand_ind = np.random.choice(np.arange(timeseries.shape[0]), 6, replace=False)
# draw one line
ln1, = ax.plot(timeseries[rand_ind[0],::scale_ind]/scale, c='blue', lw=1.0)
ln2, = ax.plot(timeseries[rand_ind[1],::scale_ind]/scale, c='red', lw=1.0)
ln3, = ax.plot(timeseries[rand_ind[2],::scale_ind]/scale, c='green', lw=1.0)
ln4, = ax.plot(timeseries[rand_ind[3],::scale_ind]/scale, c='purple', lw=1.0)
ln5, = ax.plot(timeseries[rand_ind[4],::scale_ind]/scale, c='orange', lw=1.0)
ln6, = ax.plot(timeseries[rand_ind[5],::scale_ind]/scale, c='pink', lw=1.0)
ax.vlines(np.where(years_cut[:-1] != years_cut[1:])[0],0,math.ceil(ax.get_ylim()[1]),colors='black', linewidth=0.1)

leg = ax.legend(categories[rand_ind], markerscale=1)
# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(2.0)

# helper function for the formatter
def listifed_formatter(x, pos=None):
    try:
        return time_arr_cut[int(x)]
    except IndexError:
        return ''

# make and use the formatter
mt = mticker.FuncFormatter(listifed_formatter)
ax.xaxis.set_major_formatter(mt)

# set the default ticker to only put ticks on the integers
loc = ax.xaxis.get_major_locator()
loc.set_params(integer=True)

# rotate the labels
[lab.set_rotation(15) for lab in ax.get_xticklabels()]
plt.show()

In [70]:
scale = 100

week_interval = 60*60*24*7
start_date = np.random.randint(timeseries_len//4*3,timeseries_len-week_interval//time_interval)

# make figure + axes
fig, ax = plt.subplots(tight_layout=True)
ax.set_xlabel('date')
ax.set_ylabel(f'count (x{scale})')
plt.title("Hourly Google Reviews by Category")

rand_ind = np.random.choice(np.arange(timeseries.shape[0]), 6, replace=False)
# draw one line
ln1, = ax.plot(timeseries[rand_ind[0], start_date:start_date+week_interval//time_interval]/scale, c='blue', lw=0.75, label=df.iloc[rand_ind[0], 0])
ln2, = ax.plot(timeseries[rand_ind[1], start_date:start_date+week_interval//time_interval]/scale, c='red', lw=0.75, label=df.iloc[rand_ind[1], 0])
ln3, = ax.plot(timeseries[rand_ind[2], start_date:start_date+week_interval//time_interval]/scale, c='green', lw=0.75, label=df.iloc[rand_ind[2], 0])
ln4, = ax.plot(timeseries[rand_ind[3], start_date:start_date+week_interval//time_interval]/scale, c='purple', lw=0.75, label=df.iloc[rand_ind[3], 0])
ln5, = ax.plot(timeseries[rand_ind[4], start_date:start_date+week_interval//time_interval]/scale, c='orange', lw=0.75, label=df.iloc[rand_ind[4], 0])
ln6, = ax.plot(timeseries[rand_ind[5], start_date:start_date+week_interval//time_interval]/scale, c='pink', lw=0.75, label=df.iloc[rand_ind[5], 0])


cut_time_arr = time_arr[start_date:start_date+week_interval//time_interval]
cut_years = years[start_date:start_date+week_interval//time_interval]
cut_months = months[start_date:start_date+week_interval//time_interval]
# helper function for the formatter
def listifed_formatter(x, pos=None):
    try:
        return cut_time_arr[int(x)]
    except IndexError:
        return ''
    
cut_weekdays = weekdays_timestamps[start_date:start_date+week_interval//time_interval]
Z = np.repeat(cut_weekdays.reshape(1,len(cut_weekdays)),int(ax.get_ylim()[1]+1),axis=0).reshape(int(ax.get_ylim()[1]+1),len(cut_weekdays)) 
cvals = [0,1,2,3,4,5,6]
colors = ['y','y','y','y','y','b','b']
norm=plt.Normalize(min(cvals),max(cvals))
tuples = list(zip(map(norm,cvals), colors))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", tuples)
c = ax.pcolor(Z, alpha=0.1, edgecolor=None, snap=True, cmap=cmap) 
ax.vlines(np.where(cut_weekdays[:-1] != cut_weekdays[1:])[0]+1,0,ax.get_ylim()[1], colors='black', linewidth=0.1)

# make and use the formatter
mt = mticker.FuncFormatter(listifed_formatter)
ax.xaxis.set_major_formatter(mt)

# set the default ticker to only put ticks on the integers
loc = ax.xaxis.get_major_locator()
loc.set_params(integer=True)
plt.legend()

# rotate the labels
[lab.set_rotation(15) for lab in ax.get_xticklabels()]
plt.show()